In [ ]:
import pandas as pd
import numpy as np

from autoencoder import ToyDetectors, DEDXEncoder

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow.keras.backend as K

In [ ]:
TD = ToyDetectors()
DFTD = TD.GenerateToyParticles()

In [ ]:
DFTD.head()

In [ ]:
DEDXEnc = DEDXEncoder()

In [ ]:
DEDXEnc.CreateModel()

In [ ]:
DEDXEnc.model.summary()

In [ ]:
train, test =train_test_split(DFTD, test_size=0.5)
train_data = train[["ITS", "TOF", "TPCROC0", "TPCROC1", "TPCROC2", "TRD","p"]]
test_data  = test[["ITS", "TOF", "TPCROC0", "TPCROC1", "TPCROC2", "TRD","p"]]

train_fdata= train[["ITS", "TOF", "TPCROC0", "TPCROC1", "TPCROC2", "TRD"]]
test_fdata = test[["ITS", "TOF", "TPCROC0", "TPCROC1", "TPCROC2", "TRD"]]

In [ ]:
scaler = StandardScaler()
scaler.fit(train[["ITS","TOF", "TPCROC0", "TPCROC1", "TPCROC2", "TRD","p"]])


In [ ]:
DEDXEnc.model.fit(scaler.transform(train_data),train_fdata, epochs=12, batch_size=128, 
          validation_data=[scaler.transform(test_data),test_fdata])

In [ ]:
DEDXEnc.BGfromModel([train[["ITS","TOF", "TPCROC0", "TPCROC1", "TPCROC2", "TRD","p"]]])

In [ ]:
out=DEDXEnc.model.predict(scaler.transform(test_data))
AE_predict = pd.DataFrame(out)
AE_predict.columns = ["ITS_ae", "TOF_ae", "TPCROC0_ae", "TPCROC1_ae", "TPCROC2_ae", "TRD_ae"]
ogb=DEDXEnc.BGfromModel([(scaler.transform(test_data))])
#print(ogb[-1])
GB_predict = pd.DataFrame(ogb[-1])
GB_predict.columns = ["gb_ae"]
test = test.reset_index()
df_test = pd.concat([test,AE_predict,GB_predict], axis = 1)
df_test.head()